## UNHR

In [1]:
%load_ext autoreload
%autoreload 2

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import tqdm
import os
import pandas as pd


from src.preprocessing.hatespeech_dataset_querying import hatespeech_v2_load_train_and_validation_set, map_predicted_to_label_hatesv2, hatespeech_v2_load_test_set
from src.preprocessing.toxigen_querying import load_toxigen_train_and_validation

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../..")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"## UNHR

C:\Users\Zan\Desktop\NTNU\TDT4310_Intelligent_text_analysis_language_understanding\ITAaLU_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\Zan\Desktop\NTNU\TDT4310_Intelligent_text_analysis_language_understanding\ITAaLU_project\notebook\data_exploration
C:\Users\Zan\Desktop\NTNU\TDT4310_Intelligent_text_analysis_language_understanding\ITAaLU_project


In [2]:
train_df, validation_df = hatespeech_v2_load_train_and_validation_set()
test_df = hatespeech_v2_load_test_set()
print(len(test_df))

13720


In [3]:
%%time
from src.preprocessing.general_preproc import preprocess_text


tokenizer = AutoTokenizer.from_pretrained("unhcr/hatespeech-detection")
model = AutoModelForSequenceClassification.from_pretrained("unhcr/hatespeech-detection")
# Use a pipeline as a high-level helper
pipe = pipeline("text-classification", model="unhcr/hatespeech-detection", device="cuda:0")  # 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Zan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CPU times: total: 2.64 s
Wall time: 2.72 s


In [4]:
%%time
test_df_modified = list(preprocess_text(test_df["text"]).values)
y_pred = pipe(test_df_modified)

CPU times: total: 3min 16s
Wall time: 4min 34s


In [5]:
from sklearn.metrics import classification_report
# tokenizer.to("cuda:0")
y_truth = test_df["label"]
mapped_pred = map_predicted_to_label_hatesv2(y_pred)

print(classification_report(y_truth, mapped_pred, digits=3))

              precision    recall  f1-score   support

           0      0.940     0.871     0.904     10873
           1      0.654     0.595     0.623      2520
           2      0.086     0.358     0.139       327

    accuracy                          0.808     13720
   macro avg      0.560     0.608     0.556     13720
weighted avg      0.867     0.808     0.834     13720



## Retraining the whole model for our dataset

In [6]:
# prepare the dataset in the correct format
import datasets

def convert_train_and_validation_to_dataset(train_df, validation_df):
    train_dataset = datasets.Dataset.from_pandas(df=train_df.iloc[:100], split="train")
    validation_dataset = datasets.Dataset.from_pandas(df=validation_df.iloc[:100], split="validation")
    ds = datasets.DatasetDict()
    ds["train"] = train_dataset
    ds["validation"] = validation_dataset
    return ds

hate_dataset_dict = convert_train_and_validation_to_dataset(train_df, validation_df)
hate_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

In [7]:
"""
# DEPRECATED 
# imports needed for retraining 
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

torch.cuda.empty_cache()


tokenizer = AutoTokenizer.from_pretrained("unhcr/hatespeech-detection")
model = AutoModelForSequenceClassification.from_pretrained("unhcr/hatespeech-detection")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenized_datasets = hate_dataset_dict.map(tokenize_function, batched=True)
training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=15)
metric = evaluate.load("accuracy")

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(10))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.predict()
""";

## IMSyPP model

In [8]:
%%time
model_name_pp = "IMSyPP/hate_speech_en"
# Load model directly
# tokenizer_pp = AutoTokenizer.from_pretrained(model_name_pp)
# model_pp = AutoModelForSequenceClassification.from_pretrained(model_name_pp)
# model = AutoModelForSequenceClassification.from_pretrained(model_name_pp, um_labels=3,  ignore_mismatched_sizes=True)

# Use a pipeline as a high-level helper
pipe = pipeline("text-classification", model=model_name_pp, device="cuda:0")  # 

test_df_modified = list(preprocess_text(test_df["text"]).values)
y_pred_pp = pipe(test_df_modified)

CPU times: total: 3min 21s
Wall time: 4min 35s


In [9]:
def map_predicted_to_label_hatesv2_pp(y_pred):
    y_pred_mapped = []
    for json_pair in y_pred:
        if json_pair["label"] == "LABEL_0":
            y_pred_mapped.append(0)
        elif json_pair["label"] == "LABEL_1":
            y_pred_mapped.append(0)
        elif json_pair["label"] == "LABEL_2":
            y_pred_mapped.append(1)
        elif json_pair["label"] == "LABEL_3":
            y_pred_mapped.append(2)
    return y_pred_mapped

y_pred_pp_v2 = map_predicted_to_label_hatesv2_pp(y_pred_pp)

In [10]:
from sklearn.metrics import classification_report

y_truth = test_df["label"]
print(classification_report(y_truth, y_pred_pp_v2, digits=3))

              precision    recall  f1-score   support

           0      0.932     0.846     0.887     10873
           1      0.524     0.757     0.619      2520
           2      0.105     0.067     0.082       327

    accuracy                          0.811     13720
   macro avg      0.520     0.557     0.529     13720
weighted avg      0.837     0.811     0.818     13720

